In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#| default_exp inspector

# inspector -- An inspector panel for a map.

> A simple API for displaying an inspector panel that shows results of querying map data.

In [ ]:
#|hide
from nbdev.showdoc import *
from earthengine_jupyter.map import *

In [ ]:
#|export
from collections import OrderedDict
import ee
import ipytree

In [ ]:
#|export

# Map scale at Level 0 in meters/pixel
SCALE_LEVEL_0 = 156543.03392

class MapInspector(ipytree.Tree):
    """Class representing an inspector tool that responds to map events."""
    
    def __init__(self, map_object=None, *args, **kwargs):

        point_folder = ipytree.Node('Point', icon='map')
        pixels_folder = ipytree.Node('Pixels', icon='archive')
        objects_folder = ipytree.Node('Objects', icon='archive')
        
        self.map_object = map_object
        self.layout.width = '40%'
        self.layout.max_height = '400px'
        self.layout.border = 'solid'
        self.layout.overflow = 'scroll'

        super(MapInspector, self).__init__(
            nodes=[point_folder, pixels_folder, objects_folder],
            *args, 
            **kwargs)

        if map_object:
            self.set_map(map_object)
            
        self.update_inspector()
    
    @property
    def point_node(self):
        return self.nodes[0]
    
    @point_node.setter
    def point_node(self, new_point_node):
        #(lat, lon) = new_coords
        _temp_nodes = list(self.nodes)
        _temp_nodes[0] = new_point_node
        self.nodes = _temp_nodes
    
    @property
    def pixels_node(self):
        return self.nodes[1]
    
    @property
    def objects_node(self):
        return self.nodes[2]
    
    def update_inspector(self, coords=None):
        """Update information in the inspector tree."""
            
        def _order_items(item_dict, ordering_list):
            """Orders dictionary items in a specified order."""
            list_of_tuples = [(key, item_dict[key]) for key in [x for x in ordering_list if x in item_dict.keys()]]
            return dict(list_of_tuples)  
    
        def _process_info(info):
            node_list = []    
            if isinstance(info, list):
                for count, item in enumerate(info):
                    if isinstance(item, (list, dict)):
                        node_list.append(ipytree.Node(f'{count}', nodes = _process_info(item), opened=False))
                    else:
                        node_list.append(ipytree.Node(f'{count}: {item}'))
            elif isinstance(info, dict):
                for k,v in info.items():
                    if isinstance(v, (list, dict)):
                        node_list.append(ipytree.Node(f'{k}', nodes = _process_info(v), opened=False))
                    else:
                        node_list.append(ipytree.Node(f'{k}: {v}'))
            else:
                node_list.append(ipytree.Node(f'{info}'))
            return node_list
    
        # Disable the Pixels and Objects folders if the map does not have any
        # layers. This assumes the map has a single basemap layer.
        if self.map_object:
            if len(self.map_object.layers) > 1:
                self.pixels_node.disabled = False
                self.objects_node.disabled = False
            else:
                self.pixels_node.disabled = True
                self.objects_node.disabled = True
        else:
            self.pixels_node.disabled = True
            self.objects_node.disabled = True
        
        if coords:
            (lat, lon) = coords
            
            # Clear the inspector folders before recalculating outputs.
            self.point_node.nodes = []
            self.pixels_node.nodes = []
            self.objects_node.nodes = []
            
            # Update the Point folder
            point_nodes = [
                ipytree.Node(f'Longitude: {lon:.6f}'),
                ipytree.Node(f'Latitutde: {lat:.6f}'),
                ipytree.Node(
                    f'Zoom Level: {self.map_object.zoom:.0f}'
                ),
                ipytree.Node(
                    f'Scale (approx. m/px): '
                    f'{SCALE_LEVEL_0 / 2**self.map_object.zoom:.2f}'
                )
            ]
            _point_node = ipytree.Node(f'Point ({lon:.2f}, {lat:.2f})', nodes=point_nodes)
            self.point_node = _point_node

            
            # Update the Pixels folder
            pixel_nodes = []
            for layer in self.map_object.layers:
                if not layer.base:
                    ee_type = ee.Algorithms.ObjectType(layer.ee_object).getInfo()

                    if ee_type == 'Image':
                        value_dict = layer.ee_object.reduceRegion(
                                reducer=ee.Reducer.mean(),
                                geometry=ee.Geometry.Point(lon, lat),
                                scale=30,
                                bestEffort=True
                            ).getInfo()
                        num_bands = len(value_dict.keys())

                        layer_node = ipytree.Node(f'{layer.name}: Image ({num_bands} bands)')

                        has_unmasked_pixel = False
                        for bandname in layer.ee_object.bandNames().getInfo():          
                            if value_dict[bandname] is not None:
                                has_unmasked_pixel = True
                            layer_node.add_node(
                              ipytree.Node(f'{bandname}: {value_dict[bandname]}')
                            )
                    
                        if not has_unmasked_pixel:
                            layer_node.nodes = [
                                ipytree.Node(f'No unmasked pixels at clicked point.'),
                            ] 
                        pixel_nodes.append(layer_node)        
            self.pixels_node.nodes = pixel_nodes
            
            # Update the Objects folder
            object_nodes = []
            for layer in self.map_object.layers:
                if not layer.base:
                    
                    ee_type = ee.Algorithms.ObjectType(layer.ee_object).getInfo()                    
                    layer_info = layer.ee_object.getInfo()
                    
                    # Order the layer information.
                    ordering_list = ['type', 'id', 'version', 'bands', 'properties']
                    layer_info = _order_items(layer_info, ordering_list)
                    
                    layer_node = ipytree.Node(f'{layer.name}: {ee_type} ({len(layer_info["bands"])} bands)', nodes=_process_info(layer_info))
                    
                    object_nodes.append(layer_node)
                    
            self.objects_node.nodes = object_nodes
                
    
    def register_map(self, map_object):
        def handle_interaction(type, event, coordinates):
            if type == 'click':
                self.update_inspector(coordinates)
        map_object.on_interaction(handle_interaction)
            
    def set_map(self, map_object):
        self.map_object = map_object
        self.register_map(map_object)

    def get_map(self):
        return self.map_object

In [ ]:
inspector1 = MapInspector()
inspector1

MapInspector(layout=Layout(border='solid', max_height='400px', overflow='scroll', width='40%'), nodes=(Node(ic…

## Example

In [ ]:
import ee
import ipywidgets as widgets
from earthengine_jupyter.map import JupyterMap

In [ ]:
ee.Initialize()

In [ ]:
map = JupyterMap()
inspector2 = MapInspector(map_object=map)
display(
    widgets.HBox([
        map,
        inspector2
    ],
    layout=widgets.Layout(border='1px solid black')))

map.addLayer(ee.Image.pixelLonLat(), {'min':-90, 'max':90, 'opacity':0.5}, 'LonLat')
map.addLayer(
    ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_187058_20220105'),
    {'min':0, 'max':90, 'opacity':0.5},
    'Landsat')


In [ ]:
# Add another layer. Note that Inspector will not update until clicking again on the map.
map.addLayer(ee.Image(0), {'min':0, 'max':1, 'opacity':0.5}, 'Constant Image', False)

In [ ]:
# Add another map to see if they are connected.
map3 = JupyterMap()
inspector3 = MapInspector(map_object=map3)
display(widgets.HBox([map3, inspector3]))
map3.addLayer(ee.Image.pixelLonLat(), {'min':-90, 'max':90, 'opacity':0.2}, 'MyLayer3')


In [ ]:
# Link the maps together
l = widgets.link((map, 'center'), (map3, 'center'))